In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from common import *

In [3]:
import argparse
parser = argparse.ArgumentParser(description='Run Diagnosis experiments')
parser.add_argument("--data_dir", type=str, required=True)
parser.add_argument('--display', dest='display', action='store_true')
parser.add_argument("--output_dir", type=str)
parser.add_argument("--mock", dest='mock', action='store_true')

args = parser.parse_args(['--data_dir=.', '--output_dir=outputs/', '--display'])

In [4]:
from dataloaders import readmission_dataset
data = readmission_dataset(args)

INFO - 2019-02-25 14:48:12,086 - Reading Structured data ...
INFO - 2019-02-25 14:48:12,532 - Reading Notes ...
INFO - 2019-02-25 14:48:34,844 - Stratifying ...


In [5]:
from PatientVec.Experiments.training_exps import get_basic_data

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [6]:
train_data, dev_data = get_basic_data(data, truncate=90)

Pos Percentage [0.19738451]


INFO - 2019-02-25 14:48:52,352 - Maximum Sentence Length 721825.000000 , 90 percentile length 18689.400000 ... 
INFO - 2019-02-25 14:48:52,373 - Pos Percentage of remaining data ... 
INFO - 2019-02-25 14:48:52,456 - [0.18685839]


Pos Percentage [0.20937082]


INFO - 2019-02-25 14:48:55,467 - Maximum Sentence Length 588265.000000 , 90 percentile length 19174.500000 ... 
INFO - 2019-02-25 14:48:55,472 - Pos Percentage of remaining data ... 
INFO - 2019-02-25 14:48:55,479 - [0.19723338]


In [50]:
from PatientVec.Experiments.modifiable_config_exp import attention_configs

In [63]:
config = attention_configs[1](data, structured=True, args=args)
config['training_config']['common']['class_weight'] = False
config['training_config']['common']['balanced'] = True
config['exp_config']['exp_name'] += '+balanced'
config

{'model': {'type': 'seq_classifier_with_attention',
  'embedder': {'type': 'token_embedder',
   'vocab_size': 32246,
   'embed_size': 200,
   'embedding_file': './preprocess/Readmission/embedding_matrix.npy'},
  'decoder': {'num_layers': 2,
   'hidden_dims': [128, 1],
   'activations': ['tanh', 'linear']},
  'predictor': {'type': 'binary'},
  'structured': {'use_structured': True, 'structured_dim': 142},
  'encoder': {'type': 'lstm', 'hidden_size': 128},
  'attention': {'similarity': {'type': 'additive', 'hidden_size': 128}}},
 'training_config': {'type': 'Adam',
  'groups': [('.*', {'lr': 0.001, 'weight_decay': 1e-05})],
  'common': {'bsize': 32, 'class_weight': False, 'balanced': True}},
 'exp_config': {'exp_name': 'Readmission/Attention/LSTM(hs=128)+Attention(additive)(hs=128)+Structured+balanced'}}

In [64]:
from PatientVec.models.Vanilla import ClassificationTrainer as BasicCT
from PatientVec.models.Hierarchical import ClassificationTrainer as HierCT
from PatientVec.trainer import Trainer, Evaluator

In [ ]:
train_data, dev_data = get_basic_data(data, truncate=90)
trainer = Trainer(BasicCT, config, _type=data.metrics_type, display_metrics=args.display)
trainer.train(train_data, dev_data, save_on_metric=data.save_on_metric)

evaluator = Evaluator(BasicCT, trainer.model.dirname, _type=data.metrics_type, display_metrics=args.display)
_ = evaluator.evaluate(dev_data, save_results=True)
print('='*300)

Pos Percentage [0.19738451]


INFO - 2019-02-25 15:38:04,241 - Maximum Sentence Length 721825.000000 , 90 percentile length 18689.400000 ... 
INFO - 2019-02-25 15:38:04,256 - Pos Percentage of remaining data ... 
INFO - 2019-02-25 15:38:04,274 - [0.18685839]


Pos Percentage [0.20937082]


INFO - 2019-02-25 15:38:07,245 - Maximum Sentence Length 588265.000000 , 90 percentile length 19174.500000 ... 
INFO - 2019-02-25 15:38:07,250 - Pos Percentage of remaining data ... 
INFO - 2019-02-25 15:38:07,256 - [0.19723338]
INFO - 2019-02-25 15:38:07,393 - instantiating class <class 'PatientVec.models.Model.Model'> from params {'type': 'seq_classifier_with_attention', 'embedder': {'type': 'token_embedder', 'vocab_size': 32246, 'embed_size': 200, 'embedding_file': './preprocess/Readmission/embedding_matrix.npy'}, 'decoder': {'num_layers': 2, 'hidden_dims': [128, 1], 'activations': ['tanh', 'linear']}, 'predictor': {'type': 'binary'}, 'structured': {'use_structured': True, 'structured_dim': 142}, 'encoder': {'type': 'lstm', 'hidden_size': 128}, 'attention': {'similarity': {'type': 'additive', 'hidden_size': 128}}} and extras {}
INFO - 2019-02-25 15:38:07,394 - type = seq_classifier_with_attention
INFO - 2019-02-25 15:38:07,394 - instantiating class <class 'PatientVec.models.modules.

Setting Embedding


{'accuracy': 0.6019634091923248, 'roc_auc': 0.75715259664676, 'pr_auc': 0.46584018987531617}


,0,1,micro avg,macro avg,weighted avg
f1-score,0.690,0.445,0.602,0.567,0.641
precision,0.922,0.307,0.602,0.614,0.801
recall,0.551,0.810,0.602,0.680,0.602
support,5397.000,1326.000,6723.000,6723.000,6723.000


Model Saved on  roc_auc 0.75715259664676


In [ ]:
data.generate_bowder(train_data, stop_words=True, norm=None)

In [ ]:
train_data.X = data.get_vec_encoding(train_data, _type='tfidf')
dev_data.X = data.get_vec_encoding(dev_data, _type='tfidf')

In [ ]:
train_data.X = np.array(train_data.X.todense())
dev_data.X = np.array(dev_data.X.todense())

In [ ]:
from PatientVec.Experiments.configs import vector_experiment
config = vector_experiment(data)
config

In [ ]:
from trainer import Trainer, Evaluator
from models.Baseline import ClassificationTrainer as BaseCT

In [ ]:
trainer = Trainer(BaseCT, config, _type=data.metrics_type, display_metrics=args.display)
trainer.train(train_data, dev_data, save_on_metric=data.save_on_metric)

evaluator = Evaluator(BaseCT, trainer.model.dirname, _type=data.metrics_type, display_metrics=args.display)
_ = evaluator.evaluate(dev_data, save_results=True)
print('='*300)

In [ ]:
from PatientVec.models.baselines.LR import LR, LDA
from PatientVec.Experiments.hyperparam_exps import get_basic_data

train_data, dev_data = get_basic_data(data, truncate=100)
lr = LR({'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name, 'type' : 'classifier', 'norm' : 'l2'})
lr.train(train_data)
lr.evaluate(dev_data, save_results=True)

In [ ]:
lr.print_all_features(n=40)

In [ ]:
words_tf = lr.get_features(classifier=lr.tf_idf_classifier, estimator=0, n=50)

Attention Comparison
====================

In [ ]:
from PatientVec.models.Vanilla import ClassificationTrainer as BasicCT
from PatientVec.models.Hierarchical import ClassificationTrainer as HierCT
from PatientVec.trainer import Trainer, Evaluator
from PatientVec.Experiments.modifiable_config_exp import vanilla_configs, attention_configs, hierarchical_configs, structured_configs

In [ ]:
lstm_attention_config = attention_configs[2](data, structured=True, args=args)
lstm_attn_eval = Evaluator(BasicCT, get_latest_model(os.path.join('outputs', lstm_attention_config['exp_config']['exp_name'])))
dev_data = data.get_data('dev', structured=True)
dev_data = data.filter_data_length(dev_data, truncate=90)
lstm_outputs = lstm_attn_eval.evaluate(dev_data, save_results=False)
del lstm_attn_eval

In [ ]:
dev_data = data.get_data('dev', structured=True)
dev_data = data.filter_data_length(dev_data, truncate=90)

In [ ]:
from attention_comparison import get_common_attended_words
X_1 = [dev_data.X[i] for i in range(len(dev_data.y)) if dev_data.y[i] == 1]
attn_1 = [lstm_outputs['attentions'][i] for i in range(len(dev_data.y)) if dev_data.y[i] == 1]
common_words = get_common_attended_words(X_1, attn_1, n=10)

In [ ]:
words, freqs = list(zip(*common_words.most_common(50)))

In [ ]:
words = data.vocab.map_to_words(words)

In [ ]:
list(zip(words, freqs))

In [ ]:
set(words) & set(words_tf), set(words) - set(words_tf), set(words_tf) - set(words)

Saving Models
==============

In [ ]:
from common import generate_latex_tables
keys_to_use = 
generate_latex_tables(data, keys_to_use)